In [75]:
import pandas as pd
products = pd.read_csv('amazon_baby.csv')
products.head()

,name,review,rating
0,Planetwise Flannel Wipes,"These flannel wipes are OK, but in my opinion ...",3
1,Planetwise Wipe Pouch,it came early and was not disappointed. i love...,5
2,Annas Dream Full Quilt with 2 Shams,Very soft and comfortable and warmer than it l...,5
3,Stop Pacifier Sucking without tears with Thumb...,This is a product well worth the purchase. I ...,5
4,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5


In [76]:
len(products) - len(products.dropna())

1147

In [77]:
len(products)

183531

In [78]:
products = products.dropna()

In [79]:
for i in range(0,len(products)-1):
    if type(products.iloc[i]['review']) != str:
        products.iloc[i]['review'] = str(products.iloc[i]['review'])

In [80]:
def sentiment(n):
    if n > 3:
        return 1
    if n < 3:
        return -1
    if n == 3:
        return 0
products['sentiment'] = products['rating'].apply(sentiment)
products.head()

,name,review,rating,sentiment
0,Planetwise Flannel Wipes,"These flannel wipes are OK, but in my opinion ...",3,0
1,Planetwise Wipe Pouch,it came early and was not disappointed. i love...,5,1
2,Annas Dream Full Quilt with 2 Shams,Very soft and comfortable and warmer than it l...,5,1
3,Stop Pacifier Sucking without tears with Thumb...,This is a product well worth the purchase. I ...,5,1
4,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5,1


In [81]:
def combined_features(row):
    return row['name'] + ' '+ row['review']
products['all_features'] = products.apply(combined_features, axis=1)
products.head()

,name,review,rating,sentiment,all_features
0,Planetwise Flannel Wipes,"These flannel wipes are OK, but in my opinion ...",3,0,Planetwise Flannel Wipes These flannel wipes a...
1,Planetwise Wipe Pouch,it came early and was not disappointed. i love...,5,1,Planetwise Wipe Pouch it came early and was no...
2,Annas Dream Full Quilt with 2 Shams,Very soft and comfortable and warmer than it l...,5,1,Annas Dream Full Quilt with 2 Shams Very soft ...
3,Stop Pacifier Sucking without tears with Thumb...,This is a product well worth the purchase. I ...,5,1,Stop Pacifier Sucking without tears with Thumb...
4,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5,1,Stop Pacifier Sucking without tears with Thumb...


In [82]:
X = products['all_features']
y = products['sentiment']

In [83]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [84]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
ctmTr = cv.fit_transform(X_train)
X_test_dtm = cv.transform(X_test)

In [85]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
model = LogisticRegression(max_iter=14000)
model.fit(ctmTr, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=14000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [86]:
y_pred_class = model.predict(X_test_dtm)

In [87]:
accuracy_score(y_test, y_pred_class)

0.8394376699710501

In [88]:
import joblib
joblib.dump(model, "saved_model")
joblib.dump(cv, "saved_vectorizer")

['saved_vectorizer']

In [89]:
def sentiment_analysis(sentence):
    vectorized_data = cv.transform(sentence)
    result = model.predict(vectorized_data)[0]
    if result > 0:
        return "positive"
    if result < 0:
        return "negative"
    if result == 0:
        return "neutral"

In [90]:
sentence_test = "i love KFC"

sentence = []
sentence.append(sentence_test)
print(sentiment_analysis(sentence))

positive
